In [39]:
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import seaborn as sns
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [23]:
X_train_t1 = pd.read_csv('X_train_t1.csv')
X_test_t1 = pd.read_csv('X_test_t1.csv')
y_train = pd.read_csv('y_train.csv')
y_test = pd.read_csv('y_test.csv')

In [26]:
y_test

,Unnamed: 0,0
0,0,0.584886
1,1,-0.736898
2,2,-0.912099
3,3,-0.957538
4,4,-0.895561
...,...,...
27461,27461,-0.014012
27462,27462,0.768595
27463,27463,0.928880
27464,27464,0.328137


In [27]:
y_train_scaled = pd.Series(
    y_train.iloc[:, 1].to_numpy(),
    index=X_train_t1.index
)

y_test_scaled = pd.Series(
    y_test.iloc[:, 1].to_numpy(),
    index=X_test_t1.index
)

In [29]:
X_train_t1

,Unnamed: 0,date,city,FIRE_FINAL_SIZE,FIRE_WEATHER_INDEX,FIRE_START_DATE,FIRE_OUT_DATE,FIRE_TYPE_CODE,FIRE_GENERAL_CAUSE_CODE,FIRE_RESPONSE_OBJ_CODE,...,is_active,wind_dryness_index,fire_weather_risk,temp_range,inversion_risk,weather_cluster,distance_to_cluster_0,distance_to_cluster_1,distance_to_cluster_2,distance_to_cluster_3
0,0,2017-01-01,0,-0.147757,-0.143624,1970-01-01,1970-01-01,0,5,5,...,0,-0.317433,-0.131326,5.2,-0.425077,2,4.640426,3.503436,1.683840,5.872704
1,1,2017-01-02,0,-0.147757,-0.143624,1970-01-01,1970-01-01,0,5,5,...,0,-1.493584,0.347583,6.9,-0.425077,1,5.159362,3.297060,3.676842,5.910075
2,2,2017-01-03,0,-0.147757,-0.143624,1970-01-01,1970-01-01,0,5,5,...,0,-1.161016,-0.799747,4.6,2.352513,2,5.252714,5.729000,2.890494,7.945908
3,3,2017-01-04,0,-0.147757,-0.143624,1970-01-01,1970-01-01,0,5,5,...,0,-0.737127,-0.176414,4.8,2.352513,2,3.965294,4.897211,1.881052,6.551637
4,4,2017-01-05,0,-0.147757,-0.143624,1970-01-01,1970-01-01,0,5,5,...,0,1.339850,0.543121,8.2,-0.425077,2,6.058325,3.173045,3.132508,6.289692
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109863,109863,2021-10-16,37,-0.147757,-0.143624,1970-01-01,1970-01-01,0,5,5,...,0,-0.407332,0.010238,4.5,2.352513,0,2.024764,5.426643,4.337437,5.063706
109864,109864,2021-10-17,37,-0.147757,-0.143624,1970-01-01,1970-01-01,0,5,5,...,0,0.178370,1.041605,5.1,-0.425077,0,2.725438,3.462760,3.803867,3.269933
109865,109865,2021-10-18,37,-0.147757,-0.143624,1970-01-01,1970-01-01,0,5,5,...,0,0.330163,1.054938,8.3,-0.425077,0,3.004879,3.300084,4.062727,3.021101
109866,109866,2021-10-19,37,-0.147757,-0.143624,1970-01-01,1970-01-01,0,5,5,...,0,0.446630,1.329250,15.2,-0.425077,3,4.205450,3.968259,5.382406,2.124379


In [31]:
X_train_t1.drop(columns=['Unnamed: 0'], inplace=True)
X_test_t1.drop(columns=['Unnamed: 0'], inplace=True)

In [32]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

cities = X_train_t1['city'].unique()

sarimax_models = {}
sarimax_train_pred = np.zeros(len(X_train_t1))

for city in cities:
    mask = X_train_t1['city'] == city
    idx = X_train_t1[mask].index

    dates = pd.to_datetime(X_train_t1.loc[mask, 'date'])
    y_city = y_train_scaled.loc[mask]

    order_idx = dates.argsort()
    dates = dates.iloc[order_idx]
    y_city = y_city.iloc[order_idx]
    idx = idx[order_idx]

    ts = pd.Series(y_city.values, index=dates)


    model = SARIMAX(
        ts,
        order=(1,1,1),
        seasonal_order=(1,0,1,7),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    res = model.fit(disp=False, maxiter=100)

    fitted = res.fittedvalues
    sarimax_train_pred[idx] = fitted.values

    sarimax_models[city] = {
        'model': res,
        'last_value': fitted.iloc[-1]}

/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
/usr/local/lib/python3.12/dist-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dat

In [33]:
sarimax_test_pred = np.zeros(len(X_test_t1))

for city in X_test_t1['city'].unique():
    mask = X_test_t1['city'] == city
    idx = X_test_t1[mask].index

    model_data = sarimax_models.get(city)

    if model_data and model_data['model'] is not None:

        preds = model_data['model'].forecast(steps=len(idx))
        sarimax_test_pred[idx] = preds.values

In [35]:
sarimax_r2 = r2_score(y_test_scaled, sarimax_test_pred)
sarimax_rmse = np.sqrt(mean_squared_error(y_test_scaled, sarimax_test_pred))
sarimax_mae = mean_absolute_error(y_test_scaled, sarimax_test_pred)

sarimax_r2, sarimax_rmse, sarimax_mae

(-0.000633315762725184, np.float64(0.6400560074505314), 0.5359367639598716)

In [36]:
train_idx = []
val_idx = []

for city in X_train_t1['city'].unique():
    city_data = X_train_t1[X_train_t1['city'] == city].sort_values('date')
    split = int(len(city_data) * 0.85)
    train_idx.extend(city_data.index[:split])
    val_idx.extend(city_data.index[split:])

X_tr = X_train_t1.loc[train_idx]
X_val = X_train_t1.loc[val_idx]

y_tr = y_train_scaled.loc[train_idx]
y_val = y_train_scaled.loc[val_idx]

sarimax_tr_pred = sarimax_train_pred[train_idx]
sarimax_val_pred = sarimax_train_pred[val_idx]

In [37]:
def clean_features(X):
    X = X.copy()
    X = X.drop(columns=X.select_dtypes(include=['datetime64']).columns, errors='ignore')
    X = X.drop(columns=X.select_dtypes(exclude=[np.number]).columns, errors='ignore')
    return X

X_tr_xgb = clean_features(X_tr)
X_val_xgb = clean_features(X_val)
X_test_xgb = clean_features(X_test_t1)

X_tr_xgb['sarimax_pred'] = sarimax_tr_pred
X_val_xgb['sarimax_pred'] = sarimax_val_pred
X_test_xgb['sarimax_pred'] = sarimax_test_pred

In [40]:
params = {
    'objective': 'reg:squarederror',
    'max_depth': 7,
    'learning_rate': 0.05,
    'subsample': 0.85,
    'colsample_bytree': 0.85,
    'min_child_weight': 5,
    'gamma': 0.05,
    'reg_alpha': 0.05,
    'reg_lambda': 0.1,
    'tree_method': 'hist',
    'eval_metric': 'rmse',
    'random_state': 42
}

dtrain = xgb.DMatrix(X_tr_xgb, label=y_tr)
dval = xgb.DMatrix(X_val_xgb, label=y_val)

xgb_hybrid = xgb.train(
    params,
    dtrain,
    num_boost_round=2000,
    evals=[(dtrain,'train'), (dval,'val')],
    early_stopping_rounds=100,
    verbose_eval=50
)

[0]	train-rmse:0.61962	val-rmse:0.67398
[50]	train-rmse:0.43399	val-rmse:0.50800
[100]	train-rmse:0.40639	val-rmse:0.50513
[150]	train-rmse:0.39102	val-rmse:0.50655
[200]	train-rmse:0.37827	val-rmse:0.50865
[201]	train-rmse:0.37807	val-rmse:0.50861


In [41]:
dtest = xgb.DMatrix(X_test_xgb)
xgb_test_pred = xgb_hybrid.predict(dtest)

In [42]:
X_train_xgb_only = clean_features(X_train_t1)
X_test_xgb_only = clean_features(X_test_t1)

dtrain_only = xgb.DMatrix(X_train_xgb_only, label=y_train_scaled)
dtest_only = xgb.DMatrix(X_test_xgb_only)

xgb_only = xgb.train(
    params,
    dtrain_only,
    num_boost_round=1000,
    verbose_eval=False
)

xgb_only_pred = xgb_only.predict(dtest_only)

In [43]:
results = {}

results['SARIMAX'] = sarimax_test_pred
results['XGBoost'] = xgb_only_pred

weights = {
    'Hybrid_10_90': (0.1, 0.9),
    'Hybrid_20_80': (0.2, 0.8),
    'Hybrid_30_70': (0.3, 0.7),
    'Hybrid_40_60': (0.4, 0.6),
    'Hybrid_50_50': (0.5, 0.5)
}

for k, (a,b) in weights.items():
    results[k] = a * sarimax_test_pred + b * xgb_test_pred

In [44]:
summary = []

for name, pred in results.items():
    summary.append([
        name,
        r2_score(y_test_scaled, pred),
        np.sqrt(mean_squared_error(y_test_scaled, pred)),
        mean_absolute_error(y_test_scaled, pred)
    ])

results_df = pd.DataFrame(summary, columns=['Model','R2','RMSE','MAE'])
results_df.sort_values('R2', ascending=False)

,Model,R2,RMSE,MAE
1,XGBoost,0.636542,0.385752,0.293446
2,Hybrid_10_90,0.446538,0.476019,0.381367
3,Hybrid_20_80,0.418657,0.487861,0.394536
4,Hybrid_30_70,0.385326,0.501652,0.408994
5,Hybrid_40_60,0.346543,0.517236,0.424648
6,Hybrid_50_50,0.302309,0.534456,0.441311
0,SARIMAX,-0.000633,0.640056,0.535937


In [45]:
importance = xgb_hybrid.get_score(importance_type='gain')

pd.DataFrame({
    'feature': importance.keys(),
    'importance': importance.values()
}).sort_values('importance', ascending=False).head(15)

,feature,importance
45,sarimax_pred,109.995621
15,dew_point_2m_mean (°C),26.067741
18,temperature_2m_mean (°C),20.377050
26,pm25_lag_1,11.811720
6,lat,8.764925
19,temperature_2m_max (°C),8.257782
35,month,7.098908
13,relative_humidity_2m_max (%),7.020253
42,distance_to_cluster_1,5.809898
8,year,5.679981


The studied problem represents a multi-city time-series forecasting task, where the target variable exhibits strong temporal dependence, autocorrelation, and seasonal behavior. In addition to these temporal patterns, the target is influenced by multiple exogenous variables such as meteorological conditions, air pollution indicators, and spatial information. This combination of structured time dependence and nonlinear external effects makes the problem unsuitable for relying on a single modeling paradigm.

SARIMAX was employed to explicitly model the temporal structure of the data. Unlike standard machine learning models, SARIMAX accounts for autoregressive behavior, moving average components, and differencing to address non-stationarity, while also capturing weekly seasonality through its seasonal specification. Training separate SARIMAX models for each city allows the model to preserve local temporal dynamics and prevents the mixing of heterogeneous patterns across different urban environments.

Although SARIMAX effectively captures linear temporal relationships, it has limited capacity to model complex nonlinear interactions between exogenous variables and the target. To address this limitation, SARIMAX predictions were used as an additional feature within an XGBoost model. In this hybrid framework, SARIMAX acts as a temporal feature extractor, while XGBoost learns nonlinear corrections and residual structures that remain unexplained by the statistical model.

Model evaluation results show that SARIMAX alone achieves low predictive performance, indicating that temporal structure alone is insufficient to fully explain the target variability. However, when integrated into the XGBoost model, SARIMAX predictions emerge as the most important feature, demonstrating that the extracted temporal signal is highly informative when combined with nonlinear learning.

Overall, this hybrid modeling strategy leverages the complementary strengths of statistical time-series analysis and machine learning. SARIMAX provides a principled representation of temporal dynamics, while XGBoost enhances predictive accuracy by capturing complex relationships, making the approach well-suited for real-world spatiotemporal forecasting problems.

The SARIMAX model represents a classical statistical time-series approach that focuses on modeling linear temporal dependencies and seasonality. Its main strength lies in its ability to explicitly capture autocorrelation, trends, and periodic patterns using a well-defined probabilistic framework. However, SARIMAX assumes linear relationships and has limited flexibility in handling high-dimensional exogenous variables and nonlinear interactions. This limitation is reflected in its weak standalone predictive performance, despite correctly modeling the underlying temporal structure.

XGBoost, in contrast, is a powerful machine learning method designed to capture complex nonlinear relationships between predictors and the target variable. It performs particularly well when dealing with heterogeneous features such as meteorological variables, pollution indicators, and spatial attributes. However, XGBoost does not inherently model temporal dependence and relies entirely on feature engineering to incorporate time-related information. As a result, it may fail to fully exploit the sequential nature of time-series data when used alone.

The hybrid SARIMAX–XGBoost approach combines the strengths of both methods. SARIMAX is first used to extract the linear temporal signal and seasonal structure from the data, producing time-aware predictions. These predictions are then introduced as an additional feature into the XGBoost model, allowing it to learn nonlinear residual patterns and interactions that SARIMAX cannot capture. This integration enables the hybrid model to respect time dependence while maintaining high predictive flexibility.

Empirical results demonstrate that while SARIMAX alone performs poorly and XGBoost alone achieves strong results, the hybrid approach provides a more interpretable and theoretically grounded solution. The high importance assigned to the SARIMAX prediction feature within XGBoost confirms that explicit temporal modeling contributes valuable information that enhances machine learning performance.

In summary, SARIMAX is effective for capturing structured temporal behavior but lacks expressive power, XGBoost excels at nonlinear regression but ignores temporal structure, and the hybrid approach achieves a balance by combining statistical rigor with machine learning flexibility, making it particularly suitable for complex spatiotemporal forecasting tasks